In [1]:
#import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 
import numpy as np

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, GRU, Embedding # CuDNNGRU
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
mark_start = 'ssss ' #başlangıç kelimesi. Bu kelime metinde geçmeyen bir kelime olmalıdır
mark_end = ' eeee' #bitiş kelimesi

In [4]:
data_src = []
data_dest = []

In [5]:
#dosyadaki türkçe ve ingilizce kelimeleri ayırıyoruz. İngilizce source, Tükçe destination olarak belirlendi

for line in open('tur.txt', encoding='UTF-8'):
    en_text, tr_text = line.rstrip().split('\t')
    
    tr_text = mark_start + tr_text + mark_end #türkçe hedef seti olduğu için her cümleye başlangıç ve bitiş ekleniyor
    
    data_src.append(en_text)
    data_dest.append(tr_text)

In [6]:
data_src[10]

'Jump.'

In [7]:
data_dest[10]

'ssss Defol. eeee'

In [8]:
data_src[200]

'How deep?'

In [9]:
data_dest[200]

'ssss Ne kadar derin? eeee'

In [10]:
len(data_src)

473035

In [11]:
class TokenizerWrap(Tokenizer):
    def __init__(self, texts, padding, reverse=False, num_words=None):
        Tokenizer.__init__(self, num_words=num_words)
        
        self.fit_on_texts(texts) #kelime dictionary oluşturuyoruz
        
        self.index_to_word = dict(zip(self.word_index.values(), self.word_index.keys())) #kelime dict. tersini oluşturduk
        
        self.tokens = self.texts_to_sequences(texts) #kelimeleri tokenlaştırdık
        
        if reverse:
            self.tokens = [list(reversed(x)) for x in self.tokens] #ters işaretli ise listeyi tersten oluşturuyoruz
            truncating = 'pre'
        else:
            truncating = 'post'
            
        self.num_tokens = [len(x) for x in self.tokens] #token uzunluk dizisi oluşturduk
        self.max_tokens = np.mean(self.num_tokens) + 2 * np.std(self.num_tokens) #max token uzunluğunu ortalam ve standart sapmadan hesapladık
        self.max_tokens = int(self.max_tokens)
        
        self.tokens_padded = pad_sequences(self.tokens, #padding işlemi yaparak max'tan kısa veya uzun olanları hizalayıp tüm token uzunluklarını eşitledik
                                           maxlen=self.max_tokens,
                                           padding=padding,
                                           truncating=truncating)
        
    def token_to_word(self, token): #tokenları kelimeye çeviren fonksiyon
        word = ' ' if token == 0 else self.index_to_word[token]
        return word
    
    def tokens_to_string(self, tokens): #tokenlardan metin elde ediyoruz
        words = [self.index_to_word[token] for token in tokens if token != 0]
        text = ' '.join(words)
        return text
    
    def text_to_tokens(self, text, padding, reverse=False): #metinden tokenlar çıkartıyoruz
        tokens = self.texts_to_sequences([text]) #cümleyi tokenlarına ayırıyoruz
        tokens = np.array(tokens) 
        
        if reverse:
            tokens = np.flip(tokens, axis=1) #reverse ise tokenları ters çevirdik(axis 1 satır işlemi için)
            truncating = 'pre'
        else:
            truncating = 'post'
            
        tokens = pad_sequences(tokens,
                               maxlen=self.max_tokens,
                               padding=padding,
                               truncating=truncating)
        
        return tokens

In [12]:
tokenizer_src = TokenizerWrap(texts=data_src, #ingilizce metin tanınıp anlam vektörü oluşturulacağı için (encoding)
                              padding='pre', #padding pre yani kısa olan cümlelerin başı 0 ile dolduruluyor
                              reverse=True, #böylece y.sinir ağı sonrakileri daha etkili aldığından bu işlem yapılıyor
                              num_words=None)

In [13]:
tokenizer_dest = TokenizerWrap(texts=data_dest, #türkçe metin ise gelen ingizceden türetilmiş anlam vektöründen 
                              padding='post', #yorumlandığı için daha kısa olan türkçe kelimelerin sonu 0 ile pad ediliyor (post)
                              reverse=False, #çünkü bu işlemin baş kısmı daha önemli
                              num_words=None)

In [14]:
tokens_src = tokenizer_src.tokens_padded #pad edilmiş tokenlar olarak değişkenleri belirliyoruz
tokens_dest = tokenizer_dest.tokens_padded
print(tokens_src.shape)
print(tokens_dest.shape)

(473035, 11)
(473035, 10)


In [15]:
tokens_dest[14]

array([  1,  12, 693,   2,   0,   0,   0,   0,   0,   0])

In [16]:
tokenizer_dest.tokens_to_string(tokens_dest[14])

'ssss onu yap eeee'

In [17]:
tokens_src[14]

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0, 15,  9])

In [18]:
tokenizer_src.tokens_to_string(tokens_src[14])

'it do'

In [19]:
data_src[200]

'How deep?'

In [20]:
token_start = tokenizer_dest.word_index[mark_start.strip()]
token_start

1

In [21]:
token_end = tokenizer_dest.word_index[mark_end.strip()]
token_end

2

In [22]:
encoder_input_data = tokens_src #encoder giriş için ingilizce tokenlar seçildi

In [23]:
decoder_input_data = tokens_dest[:, :-1] #dekoder girişi için türkçe tokenlar seçildi
decoder_output_data = tokens_dest[:, 1:] #dekoder çıkışı olarak ise türkçe tokenların bir basamak sonrası seçildi
#böylece y.sinir ağı gelen türkçe kelimeden sonra gelecek kelimeyi ingilizce y.sinir ağından gelen anlam vektörü ile birlikte
#yorumlayıp bulmaya çalışacak. Yani her defasında 
# (ing. anlam vektörü)+(türkçe kelime)=(türkçe sonraki kelime) şeklinde tahminde bulunmaya sistemi zorluyoruz.

In [24]:
encoder_input_data[10]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 1977])

In [25]:
decoder_input_data[200]

array([   1,   10,   20, 1355,    2,    0,    0,    0,    0])

In [26]:
decoder_output_data[200]

array([  10,   20, 1355,    2,    0,    0,    0,    0,    0])

In [27]:
tokenizer_dest.tokens_to_string(decoder_input_data[200])

'ssss ne kadar derin eeee'

In [28]:
tokenizer_dest.tokens_to_string(decoder_output_data[200])

'ne kadar derin eeee'

In [29]:
num_encoder_words = len(tokenizer_src.word_index)+3
num_decoder_words = len(tokenizer_dest.word_index)+3

In [30]:
num_encoder_words

21318

In [31]:
num_decoder_words

94061

In [32]:
embedding_size = 100

In [33]:
word2vec = {}
with open('glove.6B.100d.txt', encoding='UTF-8') as f: #6milyar ing kelime ile eğitilmiş glove veri setini yükledik
    for line in f:
        values = line.split() #boşluktan ayırdık
        word = values[0] #ilk eleman kelime
        vec = np.asarray(values[1:], dtype='float32') #ikinci eleman 100 boyutlu kelime vektörü
        word2vec[word] = vec #kelime ve kelime vektörünü dict. yükledik

In [34]:
embedding_matrix = np.random.uniform(-1, 1, (num_encoder_words, embedding_size)) #rastgele değerlerdenoluşan giriş matrisimizi oluşturduk
for word, i in tokenizer_src.word_index.items():
    if i < num_encoder_words:
        embedding_vector = word2vec.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector #eğer kelime glove setinde varsa oradaki vektörü aldık yoksa rastgele değerde kaldı
            #bu rastgele değerler eğitim esnasında hali hazırda güncellencektir.

In [35]:
embedding_matrix.shape

(21318, 100)

In [36]:
encoder_input = Input(shape=(None,), name='encoder_input') #y.sinir ağının giriş katmanını oluşturduk

In [37]:
encoder_embedding = Embedding(input_dim=num_encoder_words, #y.sinir ağınıın embedding katmanını oluşturuyoruz.
                              output_dim=embedding_size,
                              weights=[embedding_matrix],
                              trainable=True, #True olduğu için kelime vektörleri eğitilecek
                              name='encoder_embedding')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [38]:
state_size = 64 #her katmandaki nöron sayısı. (Hafıza problemi varsa küçültülür [normalde 256])

In [39]:
encoder_gru1 = GRU(state_size, name='encoder_gru1', return_sequences=True) #3 adet gru katmanları
encoder_gru2 = GRU(state_size, name='encoder_gru2', return_sequences=True)
encoder_gru3 = GRU(state_size, name='encoder_gru3', return_sequences=False) #son katman tekbir çıkış (düşünce vektörü) üreteceği için False yaptık

In [40]:
def connect_encoder(): #y.sinir ağı katmanlarını birbirine bağlıyoruz
    net = encoder_input
    
    net = encoder_embedding(net)
    
    net = encoder_gru1(net)
    net = encoder_gru2(net)
    net = encoder_gru3(net)
    
    encoder_output = net
    
    return encoder_output

In [41]:
encoder_output = connect_encoder()

In [42]:
decoder_initial_state = Input(shape=(state_size,), name='decoder_initial_state') #encoder çıkışını alacakak decoder girişi

In [43]:
decoder_input = Input(shape=(None,), name='decoder_input') #türkçe kelimlerin girileceği decoder girişi

In [44]:
decoder_embedding = Embedding(input_dim=num_decoder_words, #elimizde türkçe eğitilmiş hazır kelime vektörleri bulunmadığından
                              output_dim=embedding_size, #rastgele sayılarla vektör oluşturuyoruz. Bu vektörler zamanla eğitilecek
                              name='decoder_embedding')

In [45]:
decoder_gru1 = GRU(state_size, name='decoder_gru1', return_sequences=True) #gpu varsa gru yerine gpu versiyonu kullanılabilir
decoder_gru2 = GRU(state_size, name='decoder_gru2', return_sequences=True)
decoder_gru3 = GRU(state_size, name='decoder_gru3', return_sequences=True) #çıkışta cümle alacağımız için son katman yine sequence

In [46]:
decoder_dense = Dense(num_decoder_words,
                      activation='linear', #loss işleminde softmax uygulandığı için burada liear kullanıldı
                      name='decoder_output') #vektörü kelimeye çevirmek için one-shot vektör çıkışı elde etmek için Dense katmanı ekledik

In [47]:
def connect_decoder(initial_state): #katmanları birbirine bağlıyoruz
    net = decoder_input
    
    net = decoder_embedding(net)
    
    net = decoder_gru1(net, initial_state=initial_state)
    net = decoder_gru2(net, initial_state=initial_state)
    net = decoder_gru3(net, initial_state=initial_state)
    
    decoder_output = decoder_dense(net)
    
    return decoder_output

In [48]:
decoder_output = connect_decoder(initial_state=encoder_output) #düşünce vektörünü giriş olarak verdik

model_train = Model(inputs=[encoder_input, decoder_input], outputs=[decoder_output]) 
#girişi encoder ve dekoder input çıkışı decoder output olan eğitim modeli oluşturduk

In [49]:
model_encoder = Model(inputs=[encoder_input], outputs=[encoder_output]) #test işleminde kullanılacak modelimiz

In [50]:
decoder_output = connect_decoder(initial_state=decoder_initial_state) 

model_decoder = Model(inputs=[decoder_input, decoder_initial_state], outputs=[decoder_output])

In [51]:
def sparse_cross_entropy(y_true, y_pred): #loss fonksiyonumuzu uyguluyoruz
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred)
    loss_mean = tf.reduce_mean(loss)
    return loss_mean

In [52]:
optimizer = RMSprop(lr=1e-3) #optimizer olarak RMS kullandık

C:\Users\ysfbil\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\optimizers\optimizer_v2\rmsprop.py:135: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [53]:
decoder_target = tf.placeholder(dtype='int32', shape=(None,None)) #boyut sorunlarından dolayı placeholder kullanıyoruz

In [54]:
model_train.compile(optimizer=optimizer, #model eğitimi
                    loss=sparse_cross_entropy,
                    target_tensors=[decoder_target])

In [55]:
path_checkpoint = 'checkpoint.keras' #daha sonra kullanabilmek için eğitilen ağırlıkları kaydediyoruz
checkpoint = ModelCheckpoint(filepath=path_checkpoint, save_weights_only=True)

In [56]:
try:
    model_train.load_weights(path_checkpoint)
except Exception as error:
    print('Checkpoint yüklenirken hata oluştu. Eğitime sıfırdan başlanıyor.')
    print(error)

Checkpoint yüklenirken hata oluştu. Eğitime sıfırdan başlanıyor.
[Errno 2] Unable to open file (unable to open file: name = 'checkpoint.keras', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)


In [57]:
x_data = {'encoder_input': encoder_input_data, 'decoder_input': decoder_input_data}

In [58]:
y_data = {'decoder_output': decoder_output_data}

In [59]:
model_train.fit(x=x_data,
                y=y_data,
                batch_size=256, #her defasında eğitilecek data miktarı
                epochs=1, #gpu kullanılınca epoch 50'ye kadar arttırılabilir. epoch sayısı modelin üst üste kaç kez eğitilieceğini gösterir
                callbacks=[checkpoint])

Train on 473035 samples
473035/473035 [==============================] - 7678s 16ms/sample - loss: 3.8352


In [60]:
def translate(input_text, true_output_text=None):
    input_tokens = tokenizer_src.text_to_tokens(text=input_text, #ingilizce metni tokenlaştırdık
                                                reverse=True,
                                                padding='pre')
    
    initial_state = model_encoder.predict(input_tokens) #encoder y.sinir ağı ile düşünce vektörünü oluşturduk
    
    max_tokens = tokenizer_dest.max_tokens
    
    decoder_input_data = np.zeros(shape=(1, max_tokens), dtype=np.int) #boş bir decoder girişi oluşturduk
    
    token_int = token_start
    output_text = ''
    count_tokens = 0
    
    while token_int != token_end and count_tokens < max_tokens:
        decoder_input_data[0, count_tokens] = token_int
        x_data = {'decoder_initial_state': initial_state, 'decoder_input': decoder_input_data}
        
        decoder_output = model_decoder.predict(x_data) #çeviri vektörünü elde ettik
        
        token_onehot = decoder_output[0, count_tokens, :] #en sondaki vektörü aldık
        token_int = np.argmax(token_onehot)
        print(token_onehot)
        sampled_word = tokenizer_dest.token_to_word(token_int) #sayısal değeri metne çevirdik
        output_text += ' ' + sampled_word #metinleri bilrleştirip cümleyi oluşturduk
        count_tokens += 1
     
    print('Input text:') #sonucu yazdırdık
    print(input_text)
    print()
    
    print('Translated text:')
    print(output_text)
    print()
    
    if true_output_text is not None:
        print('True output text:')
        print(true_output_text)
        print()

In [61]:
translate(input_text=data_src[10], true_output_text=data_dest[10])

C:\Users\ysfbil\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training_v1.py:2067: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
C:\Users\ysfbil\AppData\Local\Temp\ipykernel_5552\1195559481.py:10: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  decoder_input_data = np.zeros(shape=(1, max_tokens), dtype=np.int) #boş bir decoder girişi oluşturduk


[-0.11163598 -4.4200745   1.6251723  ... -4.411964   -4.435029
 -4.438419  ]
[-0.6494635 -3.0152292  3.8118393 ... -3.0309174 -3.0377064 -3.0382628]
[-0.47236472 -3.7450523   6.667434   ... -3.7623637  -3.7799459
 -3.7562194 ]
Input text:
Jump.

Translated text:
 onlar benim eeee

True output text:
ssss Defol. eeee



In [62]:
translate(input_text='Which road leads to the airport?')

[-0.39887315 -4.130945   -0.23645152 ... -4.1154947  -4.1081567
 -4.123064  ]
[ 0.37081522 -3.4878259  -0.69126093 ... -3.4909625  -3.4787383
 -3.4787087 ]
[-1.1003442 -4.010535   1.8317606 ... -4.019283  -4.0188384 -3.9976468]
[ 1.6391962 -3.6704965  1.9146941 ... -3.6859782 -3.6723957 -3.669701 ]
[ 1.3815145 -3.3214421  4.8338675 ... -3.3248012 -3.3236551 -3.305161 ]
Input text:
Which road leads to the airport?

Translated text:
 bu benim ve benim eeee



C:\Users\ysfbil\AppData\Local\Temp\ipykernel_5552\1195559481.py:10: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  decoder_input_data = np.zeros(shape=(1, max_tokens), dtype=np.int) #boş bir decoder girişi oluşturduk
